<a href="https://colab.research.google.com/github/bugeki/ML/blob/main/military_power_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Military Power Clustering Project
-World military power
Source : globalfirepower.com on 1st may 2020

- https://data.world/vizzup/world-military-power/workspace/file?filename=World+military+power.xlsx


# Import Libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings('ignore')
#pd.set_option('display.max_rows', 500)
from sklearn.cluster import KMeans

from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

In [5]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/ML/Ass 4/World military power.xlsx",header=[0,1])
df.head()

2020 ranking                               Airforce Strength  \
  Military Strength Military Strength Power Index Aircraft Strength   
0       Afghanistan                        1.3444       Afghanistan   
1           Albania                        2.3137           Albania   
2           Algeria                        0.4659           Algeria   
3            Angola                        0.8379            Angola   
4         Argentina                        0.6521         Argentina   

                                                        \
  Aircraft Strength value Fighter/Interceptor Strength   
0                   260.0                  Afghanistan   
1                    19.0                      Albania   
2                   551.0                      Algeria   
3                   295.0                       Angola   
4                   227.0                    Argentina   

                                                               \
  Fighter/Interceptor Strength value Attack Aircraft Strength   
0                                0.0              Afghanistan   
1                                0.0                  Albania   
2                              103.0                  Algeria   
3                               72.0                   Angola   
4                               24.0                Argentina   

                                                                    \
  Attack Aircraft Strength value Transport Aircraft Fleet Strength   
0                           25.0                       Afghanistan   
1                            0.0                           Albania   
2                           22.0                           Algeria   
3                           18.0                            Angola   
4                            7.0                         Argentina   

                                           ...         Manpower  \
  Transport Aircraft Fleet Strength value  ... Total Population   
0                                    30.0  ...      Afghanistan   
1                                     0.0  ...          Albania   
2                                    59.0  ...          Algeria   
3                                    30.0  ...           Angola   
4                                     9.0  ...        Argentina   

                                      Geography                               \
  Total Population value Total Square Land Area Total Square Land Area value   
0            3,49,40,837            Afghanistan                     6,52,230   
1              30,57,220                Albania                      28748.0   
2            4,16,57,488                Algeria                    23,81,741   
3            3,03,55,880                 Angola                    12,46,700   
4            4,46,94,198              Argentina                    27,80,400   

                                                           \
  Total Coastline Coverage Total Coastline Coverage value   
0              Afghanistan                            0.0   
1                  Albania                          362.0   
2                  Algeria                          998.0   
3                   Angola                         1600.0   
4                Argentina                         4989.0   

                                                                               \
  Total Waterway Coverage Total Waterway Coverage value Total Border Coverage   
0             Afghanistan                        1200.0           Afghanistan   
1                 Albania                          41.0               Albania   
2                 Algeria                           0.0               Algeria   
3                  Angola                        1300.0                Angola   
4               Argentina                       11000.0             Argentina   

                               
  Total Border Coverage value  
0                      5987.0  
1                       691.0  


# Ingest Data  

In [13]:
df.describe()

2020 ranking       Airforce Strength  \
      Military Strength Power Index Aircraft Strength value   
count                    138.000000              138.000000   
mean                       1.460716              388.471014   
std                        1.324018             1231.981859   
min                        0.060600                0.000000   
25%                        0.574800               33.000000   
50%                        1.034100              111.000000   
75%                        2.021525              290.250000   
max                       10.168100            13264.000000   

                                                                         \
      Fighter/Interceptor Strength value Attack Aircraft Strength value   
count                         138.000000                     138.000000   
mean                           81.565217                      25.760870   
std                           230.324777                      94.528222   
min                             0.000000                       0.000000   
25%                             0.000000                       0.000000   
50%                            17.000000                       0.000000   
75%                            60.500000                      15.750000   
max                          2085.000000                     742.000000   

                                                                            \
      Transport Aircraft Fleet Strength value Trainer Aircraft Fleet value   
count                              138.000000                   138.000000   
mean                                30.231884                    82.833333   
std                                 92.330436                   240.803721   
min                                  0.000000                     0.000000   
25%                                  3.000000                     4.250000   
50%                                  9.000000                    26.000000   
75%                                 26.000000                    79.250000   
max                                945.000000                  2643.000000   

                                                                              \
      Helicopter Fleet Strength value Attack Helicopter Fleet Strength value   
count                      138.000000                             138.000000   
mean                       154.065217                              25.623188   
std                        520.183631                              97.326091   
min                          0.000000                               0.000000   
25%                         15.250000                               0.000000   
50%                         44.000000                               2.000000   
75%                        126.750000                              17.750000   
max                       5768.000000                             967.000000   

            Land Strength                         \
      Tank Strength value AFV/APC Strength value   
count          138.000000             138.000000   
mean           646.565217            2485.695652   
std           1515.463683            5410.546392   
min              0.000000               0.000000   
25%             19.250000             250.250000   
50%            153.000000             734.500000   
75%            436.250000            2000.000000   
max          12950.000000           39253.000000   

                                                                              \
      Self-Propelled Artillery Strength value Towed Artillery Strength value   
count                              138.000000                     138.000000   
mean                               212.159420                     393.978261   
std                                688.028987                     805.781703   
min                                  0.000000                       0.000000   
25%                                  0.000000             

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 60 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   (2020 ranking, Military Strength)                             138 non-null    object 
 1   (2020 ranking, Military Strength Power Index)                 138 non-null    float64
 2   (Airforce Strength, Aircraft Strength)                        138 non-null    object 
 3   (Airforce Strength, Aircraft Strength value)                  138 non-null    float64
 4   (Airforce Strength, Fighter/Interceptor Strength)             138 non-null    object 
 5   (Airforce Strength, Fighter/Interceptor Strength value)       138 non-null    float64
 6   (Airforce Strength, Attack Aircraft Strength)                 138 non-null    object 
 7   (Airforce Strength, Attack Aircraft Strength value)           138 non-n

# EDA

In [10]:
df = df.rename(columns={'Military Strength': 'Country'})

In [11]:
df.shape

(138, 60)

In [14]:
df.isna().sum().any()

True

In [15]:
df.duplicated().sum()

0

In [16]:

df.columns

MultiIndex([(     '2020 ranking',                                 'Country'),
            (     '2020 ranking',           'Military Strength Power Index'),
            ('Airforce Strength',                       'Aircraft Strength'),
            ('Airforce Strength',                 'Aircraft Strength value'),
            ('Airforce Strength',            'Fighter/Interceptor Strength'),
            ('Airforce Strength',      'Fighter/Interceptor Strength value'),
            ('Airforce Strength',                'Attack Aircraft Strength'),
            ('Airforce Strength',          'Attack Aircraft Strength value'),
            ('Airforce Strength',       'Transport Aircraft Fleet Strength'),
            ('Airforce Strength', 'Transport Aircraft Fleet Strength value'),
            ('Airforce Strength',                  'Trainer Aircraft Fleet'),
            ('Airforce Strength',            'Trainer Aircraft Fleet value'),
            ('Airforce Strength',               'Helicopter Flee

## Implement basic steps to see how is your data looks like

## Convert multi index columns to one level

## Detect and drop duplicated columns

## Check for missing values 

## Deal with missing values 
* Hint-1, use wikipedia to fill in missing values ​​comparing with similar countries
* Hint-2, Georeferences of countries can help you

## Fix the dtypes

## Drop the features that not affect military power
* Hint-1,  Georeferences not proper features

## Visualy inspect the some features

## Decriptive statistics

# Data Preprocessing

# Modelling

## Hopkins Test 
#### Assess the clusterability of a dataset. A score between 0 and 1, a score around 0.5 express no clusterability, a score tending to 1 express an uniform distrubution so clustering can't be useful and  0 express  not an uniform distrubution hence clusterering can be useful.

## Clustering with K-means

### Elbow Method 
### We use this method to find optimal k value by looking up to break point of graph. 

### Model Building and label visualisation

### Evaluate 

## Hierarchical Clustering

### Dendogram

### Model Building and label visualisation

### Evaluate 